In [38]:
#r "nuget: FSharp.Text.RegexProvider"

open System
open FSharp.Text

type Move = { Count : int; From : int; To : int }
type MoveRegex = FSharp.Text.RegexProvider.Regex< @"^move (?<Count>\d+) from (?<From>\d+) to (?<To>\d+)$" >
type BoxRegex = FSharp.Text.RegexProvider.Regex< @"^\[(?<Contents>[A-Z])\]?" >
let moveRegex = MoveRegex()
let boxRegex = BoxRegex()

let push (stack : ResizeArray<_>) item =
    stack.Insert(0, item)

let pop (stack : ResizeArray<_>) =
    let x = stack[0]
    stack.RemoveAt(0)
    x

let parseText (lines : string[]) =
    let header =
        lines 
        |> Seq.takeWhile (not << String.IsNullOrEmpty)
        |> List.ofSeq
        |> List.rev

    let moves = 
        lines
        |> Array.skip (header.Length + 1)
        |> Array.map (
            fun l -> 
                let m = moveRegex.TypedMatch l
                { Count = int m.Count.Value; From = int m.From.Value; To = int m.To.Value }
            )

    let stackNames = 
        let labels = List.head header
        labels.Split([|' '|], StringSplitOptions.RemoveEmptyEntries)
        |> Array.map int

    let stacks =
        stackNames
        |> Array.map (fun name -> (name, ResizeArray()))
        |> Map.ofArray

    let boxes = 
        header
        |> List.skip 1
        |> Seq.collect (
            fun row ->
                (row :> char seq)
                |> Seq.chunkBySize 4 
                |> Seq.map String
                |> Seq.zip stackNames
                |> Seq.choose (fun (n, t) -> boxRegex.TryTypedMatch t |> Option.map (fun r -> n, r.Contents.Value[0]))
            )
    
    for (stackLabel, contents) in boxes do
        push stacks[stackLabel] contents

    stacks, moves


Installed Packages FSharp.Text.RegexProvider, 2.1.0

In [28]:
#r "nuget:FsUnit"

open FsUnitTyped

let lines = 
    [|
        "    [D]    "
        "[N] [C]    "
        "[Z] [M] [P]"
        " 1   2   3 "
        ""
        "move 1 from 2 to 1"
        "move 2 from 2 to 1"
        "move 1 from 1 to 2"
    |]

let (stacks, moves) = parseText lines

moves |> shouldEqual [| { Count = 1; From = 2; To = 1 }; { Count = 2; From = 2; To = 1}; { Count = 1; From = 1; To = 2} |]
stacks[1] |> List.ofSeq |> shouldEqual [ 'N'; 'Z' ]
stacks[2] |> List.ofSeq |> shouldEqual [ 'D'; 'C'; 'M' ]
stacks[3] |> List.ofSeq |> shouldEqual [ 'P' ]

Installed Packages FsUnit, 5.1.0

In [39]:
let applyMove (stacks : Map<_, ResizeArray<_>>) (move : Move) =
    for _ = 1 to move.Count do
        pop stacks[move.From]
        |> push stacks[move.To]

let tops stacks =
    stacks 
    |> Map.map (fun _ s -> Seq.head s)
    |> Map.values
    |> Array.ofSeq
    |> String

In [43]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_05.txt")

let lines = File.ReadAllLines(sourcePath) 

let (stacks, moves) = 
    lines
    |> parseText

for move in moves do
    applyMove stacks move

let result = 
    stacks 
    |> tops

In [ ]:
printfn "Top boxes are %s" result

## Part 2

In [47]:
let applyMove9001 (stacks : Map<_, ResizeArray<_>>) (move : Move) =
    List.init move.Count (fun _ -> pop stacks[move.From]) 
    |> List.rev
    |> List.iter (push stacks[move.To])

let (stacks, moves) = 
    lines
    |> parseText

for move in moves do
    applyMove9001 stacks move

let result = 
    stacks 
    |> tops    

In [ ]:
printfn "Top boxes are %s" result